In [1]:
n = 2**10
KRR.<y> = PolynomialRing(RDF)
f = y**n + 1
KR.<x> = KRR.quotient(f)

prim_root = CDF(exp(2 * pi * i / (2*n)))

Cn = VectorSpace(CDF, n)

def embedCDF(poly):
    n = poly.parent().degree()
    return Cn([poly.lift()(prim_root**(5**i % (2*n))) for i in range(n)])

In [6]:
NCPUS=16

sigma = 3
data_sigma = 10
data_mu = 3
k = 2**4

In [7]:
def sample_esCDF():
    samples = [normalvariate(0, sigma) for _ in range(n*k)]
    return [KR(samples[i*n:(i+1)*n]) for i in range(k)]

def sample_xsCDF():
    samples = [normalvariate(data_mu, data_sigma) for _ in range(k)]
    return [KR(samples[i]).lift() for i in range(k)]

def l2_normCDF(x):
    return x.lift().norm(2)

def canl2_normCDF(x):
    return embedCDF(x).norm(2)

def inf_normCDF(x):
    return x.lift().norm(infinity)

def caninf_normCDF(x):
    return embedCDF(x).norm(infinity)

def compute_avg_eCDF(es):
    k = len(es)
    avg_e = sum(es[i] for i in range(k))
    return avg_e * (1/k)

def embed_esCDF(es):
    return [embedCDF(e) for e in es]

def embed_avg_eCDF(avg_e):
    return embedCDF(avg_e)

def inf_boundCDF(es, avg_e, xs):
    k = len(xs)
    assert k == len(es)
    n = es[0].parent().degree()
    return 4*k*sum(inf_normCDF(es[i] - avg_e) * (xs[i] + k*n*inf_normCDF(es[i])) for i in range(k))

def caninf_boundCDF(es, avg_e, xs):
    k = len(xs)
    assert k == len(es)
    n = es[0].parent().degree()
    return 4*k*sum(caninf_normCDF(es[i] - avg_e) * (xs[i] + n * k * caninf_normCDF(es[i])) for i in range(k))

def canl2_boundCDF(es, avg_e, xs):
    k = len(xs)
    assert k == len(es)
    return 4*k*sum(canl2_normCDF(es[i] - avg_e) * (xs[i] + k * caninf_normCDF(es[i])) for i in range(k))

def gauss_approxCDF(es, xs):
    k = len(xs)
    assert k == len(es)
    n = es[0].parent().degree()
    xs_sqnorm = N(sum(xs[i]**2 for i in range(k)))
    return sqrt(n)*sqrt(4*sigma**2 * (2*k+1)*(xs_sqnorm + 2*n*k*sigma**2))

def actual_errorCDF(es, avg_e, xs):
    k = len(es)
    assert k == len(xs)
    return sum(xs[i] + k*es[i]*avg_e for i in range(k))

def dataCDF(es, avg_e, xs):
    excess = lambda a,b: N(log(a/b,2))
    err = actual_errorCDF(es, avg_e, xs)
    infbd = inf_boundCDF(es, avg_e, xs)
    inf_excess = excess(infbd, inf_normCDF(err))
    cinfbd = caninf_boundCDF(es, avg_e, xs)
    caninf_excess = excess(cinfbd, caninf_normCDF(err))
    l2err = l2_normCDF(err)
    print(l2err)
    print(infbd)
    print(cinfbd)
    n = es[0].parent().degree()
    caninf_cross = excess(n * cinfbd,l2err)
    inf_cross = excess(sqrt(n) * infbd, l2err)
    canl2_excess = excess(canl2_boundCDF(es, avg_e, xs), canl2_normCDF(err))
    gauss_excess = excess(gauss_approxCDF(es, xs), l2err)
    return [inf_excess, inf_cross, caninf_excess, caninf_cross, canl2_excess, gauss_excess]

def trialCDF():
    es = sample_esCDF()
    avg_e = compute_avg_eCDF(es)
    xs = sample_xsCDF()
    return dataCDF(es, avg_e, xs)

In [8]:
%prun -s cumulative vals = trialCDF()

198779.748962257
1629859734.675626
1050765782724.4648
 

In [9]:
vals #### old below

[16.4040972091805,
 18.0012894040877,
 19.9640225954612,
 32.3337670031660,
 10.9290175735880,
 1.59583589218983]

In [ ]:
n = 2**10

NCPUS=16

KK = CyclotomicField(2*n)

zeta = KK.gen()
Cn = VectorSpace(CC, KK.degree())

In [ ]:
sigma = 3
data_sigma = 10
data_mu = 0
k = 2**4

In [ ]:
def sample_es():
    assert n * k % NCPUS == 0
    jobs_per_cpu = n * k // NCPUS
    @parallel(ncpus=NCPUS)
    def sample_batch(sigma):
        set_random_seed()
        return [normalvariate(0, sigma) for _ in range(jobs_per_cpu)]
    vals = sample_batch([sigma for _ in range(NCPUS)])
    samples = sum((val[-1] for val in vals),[])
    return [KK(samples[i*n:(i+1)*n]) for i in range(k)]

def sample_complex_es():
    # First n/2 are Re(output), second n/2 are Im(output)
    # Extend to n-dim C vector by setting output[0] = conj(output[-1])
    assert n * k % NCPUS == 0
    jobs_per_cpu = n * k // NCPUS
    @parallel(ncpus=NCPUS)
    def sample_batch(sigma):
        set_random_seed()
        return [normalvariate(0, sigma) for _ in range(jobs_per_cpu)]
    vals = sample_batch([sigma * sqrt(n) for _ in range(NCPUS)])
    samples = sum((val[-1] for val in vals),[])
    # convert n Normals to conj C^n vecs
    def convert(sample_batch):
        output = [_ for _ in range(n)]
        for i in range(n//2):
            output[i] = CC(sample_batch[i], sample_batch[i+n//2])
            output[-i+1] = output[i].conjugate()
        return Cn(output)        
    return [convert(samples[i*n:(i+1)*n]) for i in range(k)]


def sample_xs():
    samples = [normalvariate(data_mu, data_sigma) for _ in range(k)]
    return [KK(samples[i]) for i in range(k)]

In [ ]:
def l2_norm(x):
    return x.polynomial().norm(2)

def canl2_norm(x):
    return Cn(x.complex_embeddings()).norm(2)

def inf_norm(x):
    return x.polynomial().norm(infinity)

def caninf_norm(x):
    return Cn(x.complex_embeddings()).norm(infinity)

In [ ]:
def normalize_elem(elem):
    return KK(elem.polynomial().map_coefficients(numerical_approx))

def compute_avg_e(es):
    k = len(es)
    avg_e = sum(es[i] for i in range(k))
    return normalize_elem(avg_e * (1/k))

def compute_complex_avg_e(complex_es):
    return (1/len(complex_es)) * sum(complex_e for complex_e in complex_es)

def embed_es(es):
    assert k % NCPUS == 0
    jobs_per_cpu = k // NCPUS
    @parallel(ncpus=NCPUS)
    def embed_batch(batch_es):
        return [batch_es[i].complex_embeddings() for i in range(jobs_per_cpu)]
    embedded_es = embed_batch([es[i*jobs_per_cpu:(i+1)*jobs_per_cpu] for i in range(NCPUS)])
    embedded_es = sum((batch[-1] for batch in embedded_es), [])
    return [Cn(embedded_e) for embedded_e in embedded_es]

def embed_avg_e(avg_e):
    return Cn(normalize_elem(avg_e).complex_embeddings())

def inf_bound(es, avg_e, xs):
    k = len(xs)
    assert k == len(es)
    return 4*k*sum(inf_norm(es[i] - avg_e) * (xs[i] + k*n*inf_norm(es[i])) for i in range(k))

def caninf_bound(embedded_es, embedded_avg_e, xs):
    # Embed polynomials, operate on embedded vectors
    k = len(xs)
    assert k == len(embedded_es)
    n = embedded_es[0].parent().degree()
    return 4*k*sum((embedded_es[i] - embedded_avg_e).norm(infinity) * (xs[i] + n * k * embedded_es[i].norm(infinity)) for i in range(k))

def canl2_bound(embedded_es, embedded_avg_e, xs):
    # Embed polynomials, operate on embedded vectors
    k = len(xs)
    assert k == len(embedded_es)
    return 4*k*sum((embedded_es[i] - embedded_avg_e).norm(2) * (xs[i] + k * embedded_es[i].norm(infinity)) for i in range(k))

def gauss_approx(es, xs):
    k = len(xs)
    assert k == len(es)
    n = es[0].parent().degree()
    xs_sqnorm = N(sum(xs[i]**2 for i in range(k)))
    return sqrt(4*sigma**2 * (2*k+1)*(xs_sqnorm + 2*n*k*sigma**2))

def actual_error(es, avg_e, xs):
    k = len(es)
    assert k == len(xs)
    assert k % NCPUS == 0
    num_jobs = k // NCPUS
    @parallel(ncpus=NCPUS)
    def summand_batch(batch_xs, batch_es):
        return sum(normalize_elem(2*batch_xs[i]+k*batch_es[i]*avg_e) for i in range(num_jobs))
    res = summand_batch([(xs[i], es[i]) for i in range(k)])
    return normalize_elem(sum(val[-1] for val in res))

def actual_complex_error(complex_es, avg_complex_e, xs):
    k = len(complex_es)
    assert k == len(xs)
    return sum(2 * Cn(xs[i].complex_embeddings()) + k * complex_es[i].pairwise_product(avg_complex_e) for i in range(k))


def data(es, avg_e, xs, embedded_es, embedded_avg_e):
    excess = lambda a,b: log(a/b,2)
    err = actual_error(es, avg_e, xs)
    inf_excess = excess(inf_bound(es, avg_e, xs), inf_norm(err))
    caninf_excess = excess(caninf_bound(embedded_es, embedded_avg_e, xs), caninf_norm(err))
    canl2_excess = excess(canl2_bound(embedded_es, embedded_avg_e, xs), canl2_norm(err))
    gauss_excess = excess(gauss_approx(es, xs), l2_norm(err))
    return [inf_excess, caninf_excess, canl2_excess, gauss_excess]

def trial():
    es = sample_es()
    avg_e = compute_avg_e(es)
    xs = sample_xs()
    embedded_es = embed_es(es)
    embedded_avg_e = embed_avg_e(avg_e)
    return data(es, avg_e, xs, embedded_es, embedded_avg_e)

def real_trial():
    es = sample_es()
    avg_e = compute_avg_e(es)
    xs = sample_xs()
    excess = lambda a,b: log(a/b,2)
    err = actual_error(es, avg_e, xs)
    inf_excess = excess(inf_bound(es, avg_e, xs), inf_norm(err))
    gauss_excess = excess(gauss_approx(es, xs), l2_norm(err))
    return [inf_excess, gauss_excess]
    

def complex_trial():
    embedded_es = sample_complex_es()
    embedded_avg_e = compute_complex_avg_e(embedded_es)
    xs = sample_xs()
    embedded_err = actual_complex_error(embedded_es, embedded_avg_e, xs)
    excess = lambda a,b: log(a/b,2)
    caninf_excess = excess(caninf_bound(embedded_es, embedded_avg_e, xs), embedded_err.norm(infinity))
    canl2_excess = excess(canl2_bound(embedded_es, embedded_avg_e, xs), embedded_err.norm(2))
    return [caninf_excess, canl2_excess]

def synthetic_trial():
    real = real_trial()
    cmplx = complex_trial()
    return [real[0], cmplx[0], cmplx[1], real[1]]

In [ ]:
%prun -s cumulative res = trial(), synthetic_trial()

In [11]:
TRIALS = 8
k = 2**4

DATA_MUs = [3, 0]
DATA_SIGMAs = [3, 10]

for j in range(len(DATA_MUs)):
    data_mu = DATA_MUs[j]
    data_sigma = DATA_SIGMAs[j]


    with open(f"mu{data_mu}-sigma{data_sigma}-n2{log(n,2)}.txt", "a+") as outF:
        #[inf_excess, inf_cross, caninf_excess, caninf_cross, canl2_excess, gauss_excess]

        #outF.write("{inf_excess, inf_cross, caninf_excess, caninf_cross, canl2_excess, gauss_excess}"+"\n")
        for i in range(TRIALS):
            print(i)
            outF.write(str(trialCDF())+"\n")



0
200621.725993948
1799739203.0233643
1057210732028.2037
1
214391.831600318
1803027593.3573925
1030226140134.9191
2
238941.448766989
1602165727.828261
1064907193380.697
3
211800.356639338
1717790636.5622916
1011630941298.6747
4
213460.795228286
1566432451.224437
927377922347.679
5
207439.915606595
1588128085.3568578
963369934962.3986
6
224918.422735090
1791317512.6206002
1005868860232.1118
7
193022.945890993
1747052118.2554028
1007917359357.8461
0
206473.491701146
1694131950.5562913
1092834665267.9136
1
208802.610871917
1714630528.6993272
961842503168.4353
2
201210.764953750
1635386114.5899043
1024177513781.0447
3
199097.620085930
1920831920.0826812
986171225679.5803
4
226566.671195736
1702811840.9696932
982612605396.1191
5
201504.370071249
1562231416.7797663
1148523803066.4004
6
215994.970454765
1705321167.0452216
983847996448.1775
7
205679.259598611
1847062320.8128648
1043640094884.7563


In [ ]:


histogram([range(0, 1, .1)*10, [nv(0, 1) for _ in range(100)]], color=['red', 'green'], bins=5)